<a href="https://colab.research.google.com/github/daspartho/fastai-part2/blob/main/minibatch_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Get the data

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from urllib.request import urlretrieve
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

if not path_gz.exists():
    urlretrieve(MNIST_URL, path_gz)

with gzip.open(path_gz, 'rb') as f: 
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

### Initial Setup

#### Data

In [2]:
n,m = x_train.shape
c = y_train.max()+1
nh=50
n,m,c

(50000, 784, tensor(10))

In [3]:
from torch import nn

class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [
            nn.Linear(n_in, nh), 
            nn.ReLU(),
            nn.Linear(nh, n_out),
        ]
    def __call__(self, x):
        for l in self.layers:
            x = l(x)
        return x

In [4]:
model = Model(m, nh, 10)
pred = model(x_train)
pred.shape

torch.Size([50000, 10])

#### Cross entrophy loss

In [5]:
def log_softmax(x):
    return x - x.exp().sum(-1, keepdim=True).log()

In [6]:
log_softmax(pred)

tensor([[-2.36, -2.28, -2.09,  ..., -2.43, -2.47, -2.11],
        [-2.37, -2.25, -2.09,  ..., -2.46, -2.43, -2.11],
        [-2.34, -2.31, -2.14,  ..., -2.44, -2.48, -2.14],
        ...,
        [-2.26, -2.25, -2.13,  ..., -2.36, -2.53, -2.17],
        [-2.39, -2.30, -2.18,  ..., -2.38, -2.42, -2.11],
        [-2.40, -2.25, -2.14,  ..., -2.38, -2.45, -2.23]], grad_fn=<SubBackward0>)

In [7]:
def logsumexp(x):
    a = x.max(-1)[0]
    return a + (x-a.unsqueeze(dim=1)).exp().sum(-1).log()

In [8]:
from fastcore.test import test_close
test_close(logsumexp(pred), pred.logsumexp(-1)) # comparing with pytorch's implementation

In [9]:
def log_softmax(x): return x - logsumexp(x).unsqueeze(dim=-1)

In [10]:
sm_pred = log_softmax(pred)
sm_pred

tensor([[-2.36, -2.28, -2.09,  ..., -2.43, -2.47, -2.11],
        [-2.37, -2.25, -2.09,  ..., -2.46, -2.43, -2.11],
        [-2.34, -2.31, -2.14,  ..., -2.44, -2.48, -2.14],
        ...,
        [-2.26, -2.25, -2.13,  ..., -2.36, -2.53, -2.17],
        [-2.39, -2.30, -2.18,  ..., -2.38, -2.42, -2.11],
        [-2.40, -2.25, -2.14,  ..., -2.38, -2.45, -2.23]], grad_fn=<SubBackward0>)

In [11]:
y_train[:3]

tensor([5, 0, 4])

In [12]:
sm_pred[0,5],sm_pred[1,0],sm_pred[2,4]

(tensor(-2.40, grad_fn=<SelectBackward0>),
 tensor(-2.37, grad_fn=<SelectBackward0>),
 tensor(-2.14, grad_fn=<SelectBackward0>))

In [13]:
sm_pred[[0,1,2],y_train[:3]]

tensor([-2.40, -2.37, -2.14], grad_fn=<IndexBackward0>)

In [14]:
def nll(inp, targ):
    return - inp[range(targ.shape[0]), targ].mean()

In [15]:
loss = nll(sm_pred, y_train)
loss

tensor(2.30, grad_fn=<NegBackward0>)

In [16]:
from torch.nn import functional as F
test_close(F.nll_loss(F.log_softmax(pred, -1), y_train), loss, 1e-3)

In [17]:
test_close(F.cross_entropy(pred, y_train), loss, 1e-3)

### Basic training loop

In [18]:
bs = 50

xb = x_train[:bs]
yb = y_train[:bs]
preds = model(xb)
preds.shape

torch.Size([50, 10])

In [19]:
loss_fn = F.cross_entropy
loss_fn(preds, yb)

tensor(2.28, grad_fn=<NllLossBackward0>)

In [20]:
def acc(out, y):
    return (out.argmax(dim=1)==y).float().mean()
acc(preds, yb)

tensor(0.16)

In [21]:
def report(loss, accuracy):
    print(f'loss: {loss:.2f}, accuracy: {accuracy:.2f}')

In [22]:
lr = 0.5
epochs = 3

In [23]:
# basic training loop
for epoch in range(epochs):
    for i in range(0, n, bs):
        s = slice(i, min(n, bs+i))
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        with torch.inference_mode():
            for l in model.layers:
                if hasattr(l, 'weight'):
                    l.weight -= l.weight.grad*lr
                    l.bias -= l.bias.grad*lr
                    l.weight.grad.zero_()
                    l.bias.grad.zero_()
    accuracy = acc(preds, yb)
    report(loss, accuracy)

loss: 0.08, accuracy: 0.96
loss: 0.05, accuracy: 0.98
loss: 0.07, accuracy: 0.96


### Using parameters and optim

#### Parameters

In [24]:
class MLP(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [25]:
model = MLP(m, nh, 10)
model

MLP(
  (l1): Linear(in_features=784, out_features=50, bias=True)
  (l2): Linear(in_features=50, out_features=10, bias=True)
  (relu): ReLU()
)

In [26]:
model.l1

Linear(in_features=784, out_features=50, bias=True)

In [27]:
for name,l in model.named_children():
    print(f"{name}: {l}")

l1: Linear(in_features=784, out_features=50, bias=True)
l2: Linear(in_features=50, out_features=10, bias=True)
relu: ReLU()


In [28]:
for p in model.parameters():
    print(p.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


In [29]:
def fit():
    for epoch in range(epochs):
        for i in range(0, n, bs):
            s = slice(i, min(n, bs+i))
            xb, yb = x_train[s], y_train[s]
            preds = model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr   
                model.zero_grad()                 
        accuracy = acc(preds, yb)
        report(loss, accuracy)

In [30]:
fit()

loss: 0.12, accuracy: 0.96
loss: 0.09, accuracy: 0.96
loss: 0.08, accuracy: 0.94


In [31]:
class MyModule():
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self._modules = {}
        self.l1 = nn.Linear(n_in, nh)
        self.l2 = nn.Linear(nh, n_out)
        self.relu = nn.ReLU()

    def __setattr__(self, k, v):
        if not k.startswith('_'):
            self._modules[k] = v
        super().__setattr__(k, v)

    def __repr__(self):
        return f"{self._modules}"

    def parameters(self):
        for l in self._modules.values():
            yield from l.parameters()

In [32]:
module = MyModule(m, nh, 10)
module

{'l1': Linear(in_features=784, out_features=50, bias=True), 'l2': Linear(in_features=50, out_features=10, bias=True), 'relu': ReLU()}

In [33]:
for p in module.parameters():
    print(p.shape)

torch.Size([50, 784])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])


#### Registering modules

In [34]:
from functools import reduce

In [35]:
class Model(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers=layers
        for i, l in enumerate(self.layers):
            self.add_module(f"layer_{i}", l)
    def forward(self, x):
        return reduce(lambda val, layer: layer(val), self.layers, x)

In [36]:
layers = [nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10)]
model = Model(layers)
model

Model(
  (layer_0): Linear(in_features=784, out_features=50, bias=True)
  (layer_1): ReLU()
  (layer_2): Linear(in_features=50, out_features=10, bias=True)
)

In [37]:
model(xb).shape

torch.Size([50, 10])

#### nn.ModuleList

In [38]:
class SequentialModel(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
    def forward(self, x):
        for l in self.layers:
            x = l(x)
        return x

In [39]:
model = SequentialModel(layers)
model

SequentialModel(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [40]:
fit()

loss: 0.13, accuracy: 0.96
loss: 0.10, accuracy: 0.96
loss: 0.05, accuracy: 0.98


#### nn.Sequential

In [41]:
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))
model

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)

In [42]:
fit()

loss: 0.11, accuracy: 0.96
loss: 0.07, accuracy: 0.98
loss: 0.04, accuracy: 1.00


#### Optim

In [43]:
class Optimizer():
    def __init__(self, params, lr=0.5):
        self. params = list(params)
        self.lr = lr

    def step(self):
        with torch.no_grad():
            for p in self.params:
                p -= p.grad * self.lr   
    
    def zero_grad(self):
        for p in self.params:
            p.grad.zero_()   

In [44]:
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))
opt = Optimizer(model.parameters())
loss_fn(model(xb), yb)

tensor(2.29, grad_fn=<NllLossBackward0>)

In [45]:
for epoch in range(epochs):
    for i in range(0, n, bs):
        s = slice(i, min(n, bs+i))
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()               
    accuracy = acc(preds, yb)
    report(loss, accuracy)

loss: 0.15, accuracy: 0.94
loss: 0.08, accuracy: 0.96
loss: 0.06, accuracy: 0.96


In [46]:
from torch import optim

In [47]:
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))
opt = optim.SGD(model.parameters(), lr=lr) 
loss_fn(model(xb), yb)

tensor(2.28, grad_fn=<NllLossBackward0>)

In [48]:
for epoch in range(epochs):
    for i in range(0, n, bs):
        s = slice(i, min(n, bs+i))
        xb, yb = x_train[s], y_train[s]
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        opt.step()
        opt.zero_grad()               
    accuracy = acc(preds, yb)
    report(loss, accuracy)

loss: 0.16, accuracy: 0.96
loss: 0.11, accuracy: 0.96
loss: 0.05, accuracy: 0.98
